# Проектный практикум 3. Учебная задача. Ч.2 Обучение модели.

Шаги:
1. Загрузка данных
2. Токенизация данных
3. Дообучение модели
4. Сохранение модели

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

df = pd.read_csv("/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/df_prepare.csv")
# df = pd.read_csv("/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/df_prepare_30k.csv")

In [4]:
dataset = Dataset.from_pandas(df[['text']])

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [6]:
# Токенизация данных
def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding='max_length',
        max_length=512,
    )
    # Add labels that are the same as input_ids
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/498895 [00:00<?, ? examples/s]

In [7]:
# Задаём аргументы для дообучения
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    prediction_loss_only=True,
    fp16=True,
)

trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset, tokenizer=tokenizer)

<ipython-input-7-442cbe4bdf6f>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# Запуск дообучения модели
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,2.210200
200,0.501900
300,0.489600
400,0.482300
500,0.483800
600,0.482900
700,0.481300
800,0.470100
900,0.473500
1000,0.477900


Step,Training Loss
100,2.210200
200,0.501900
300,0.489600
400,0.482300
500,0.483800
600,0.482900
700,0.481300
800,0.470100
900,0.473500
1000,0.477900


TrainOutput(global_step=5844, training_loss=0.48702207124045577, metrics={'train_runtime': 12679.0249, 'train_samples_per_second': 118.044, 'train_steps_per_second': 0.461, 'total_flos': 3.90883995942912e+17, 'train_loss': 0.48702207124045577, 'epoch': 2.9985247899429157})

In [9]:
# Сохраняем модель и токенайзер в Google диск
model.save_pretrained("/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/model")
tokenizer.save_pretrained("/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/model")

('/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/model/tokenizer_config.json',
 '/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/model/special_tokens_map.json',
 '/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/model/vocab.json',
 '/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/model/merges.txt',
 '/content/drive/MyDrive/invest/my_courses/ml_urfu/3 sem/hackaton/model/added_tokens.json')